<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Leveraging Apache Spark for Smart Building HVAC Monitoring

**Estimated time needed: 30 minutes**

### Objectives

After completing this lab, you will be able to:

- Explain the distributed architecture of Spark in the context of smart building monitoring
- Simulate real-time sensor data for HVAC systems in a building
- Perform SQL queries to detect critical environmental conditions and calculate average readings
- Determine the aggregated results to the console for immediate insights into room conditions


## Background
Smart Building Solutions, Inc. specializes in optimizing HVAC (heating, ventilation, and air conditioning) systems to enhance comfort and energy efficiency in commercial buildings. By monitoring temperature and humidity levels in real-time across various rooms, the company aims to ensure optimal indoor conditions and preemptively address potential HVAC issues.

With a continuous influx of sensor data, Smart Building Solutions needs to process and analyze this data in real-time to maintain the quality of the indoor environment.

## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [8]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [9]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [10]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [11]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [12]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [14]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)






### Output the results to the console:

Display the results from each query in real-time.


In [15]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------


[Stage 2:>                (0 + 8) / 200][Stage 4:>                (0 + 0) / 200]

+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 43.34375401095521|2025-08-06 14:08:...|
|      0|       15.0| 55.17960520814723|2025-08-06 14:08:...|
|      0|       15.0| 42.93971027189441|2025-08-06 14:08:...|
|      0|       15.0|59.624455614827006|2025-08-06 14:08:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
+-------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|63.11344728933466|2025-08-06 14:08:...|
|      0|       15.0|61.02689911086321|2025-08-06 14:08:...|
|      0|       15.0|44.78791575352364|2025-08-06 14:08:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



[Stage 7:>(158 + 8) / 200][Stage 8:>    (0 + 0) / 8][Stage 9:>    (0 + 0) / 8]8]

### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [ ]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


[Stage 7:>(163 + 8) / 200][Stage 8:>    (0 + 0) / 8][Stage 9:>    (0 + 0) / 8]

********Critical Temperature Values*******


-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|   64.542453288296|2025-08-06 14:08:...|
|      0|       15.0|  68.2829336731192|2025-08-06 14:08:...|
|      0|       15.0|47.312182673670684|2025-08-06 14:08:...|
|      0|       15.0| 61.41255826049551|2025-08-06 14:09:...|
|      0|       15.0| 63.11826065364515|2025-08-06 14:09:...|
|      0|       15.0| 67.39378936964142|2025-08-06 14:09:...|
|      0|       15.0| 57.03241551075707|2025-08-06 14:08:...|
|      0|       15.0| 55.29368431065291|2025-08-06 14:08:...|
|      0|       15.0| 41.90213660330837|2025-08-06 14:08:...|
|      0|       15.0|42.044049050449416|2025-08-06 14:09:...|
|      0|       15.0| 43.97445486511934|2025-08-06 14:09:...|
|      0|       15.0| 45.1986015717

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|  69.1509250789342|2025-08-06 14:09:...|
|      0|       15.0| 69.72758010221445|2025-08-06 14:09:...|
|      0|       15.0|  58.0054647142333|2025-08-06 14:09:...|
|      0|       15.0| 69.07826668245859|2025-08-06 14:09:...|
|      0|       15.0| 47.55974706242981|2025-08-06 14:09:...|
|      0|       15.0| 54.40728334906027|2025-08-06 14:09:...|
|      0|       15.0|56.522956000453895|2025-08-06 14:09:...|
|      0|       15.0| 61.40250775327956|2025-08-06 14:09:...|
|      0|       15.0| 64.76751949137315|2025-08-06 14:09:...|
|      0|       15.0| 52.79090525508008|2025-08-06 14:09:...|
|      0|       15.0|50.315061266032224|2025-08-06 14:09:...|
|      0|       15.0| 44.5546509483

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 64.99515199100178|2025-08-06 14:09:...|
|      0|       15.0| 65.09269228951172|2025-08-06 14:09:...|
|      0|       15.0|  47.6946500407071|2025-08-06 14:10:...|
|      0|       15.0| 57.45282115197594|2025-08-06 14:09:...|
|      0|       15.0|  53.9877553845992|2025-08-06 14:09:...|
|      0|       15.0| 65.41641708438686|2025-08-06 14:09:...|
|      0|       15.0|54.004354918728296|2025-08-06 14:09:...|
|      0|       15.0| 56.42418399059093|2025-08-06 14:09:...|
|      0|       15.0| 43.93073781771707|2025-08-06 14:09:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------

...

[Stage 22:(191 + 8) / 200][Stage 23:>   (0 + 0) / 8][Stage 24:>   (0 + 0) / 8]]]

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|45.273904058499156|2025-08-06 14:10:...|
|      0|       15.0| 66.51903459726091|2025-08-06 14:10:...|
|      0|       15.0| 61.08469339148617|2025-08-06 14:10:...|
|      0|       15.0|46.799023347626154|2025-08-06 14:10:...|
|      0|       15.0|  54.2992376151091|2025-08-06 14:10:...|
|      0|       15.0| 69.87666684013965|2025-08-06 14:10:...|
|      0|       15.0| 64.40909532432143|2025-08-06 14:10:...|
|      0|       15.0|50.114740839730565|2025-08-06 14:10:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+-------------------+
|      2|33.238035715967705| 56.16008597417182|2025-08-06 14:10:00|
|      4|31.644145427514733| 53.23161136405627|2025-08-06 14:09:00|
|      4| 32.42476786212472|57.216087960433995|2025-08-06 14:08:00|
|      6|33.185535219634446| 55.18189396159054|2025-08-06 14:08:00|
|      8| 31.30979926892635| 57.35126614957439|2025-08-06 14:10:00|
|      1|29.204033307367354| 51.32651439300745|2025-08-06 14:10:00|
|      7| 31.06555531695883| 57.08781554123193|2025-08-06 14:08:00|
|      6| 34.06186993139565|57.410714720405664|2025-08-06 14:09:00|
|      0|              15.0| 50.79530845225681|2025-08-06 14:08:00|
|      7| 32.14102962900383|55.565959571443244|2025-08-06 14:09:00|
|      0|          

-------------------------------------------
Batch: 9
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 60.08563263395898|2025-08-06 14:10:...|
|      0|       15.0|45.443710936608916|2025-08-06 14:11:...|
|      0|       15.0| 68.90022352897576|2025-08-06 14:10:...|
|      0|       15.0| 42.53284262956933|2025-08-06 14:11:...|
|      0|       15.0|51.983656563950056|2025-08-06 14:10:...|
|      0|       15.0| 53.63195356344977|2025-08-06 14:11:...|
|      0|       15.0| 53.35806658244321|2025-08-06 14:11:...|
|      0|       15.0|59.999424749962145|2025-08-06 14:11:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+----

[Stage 28:>(68 + 8) / 200][Stage 29:>   (0 + 0) / 8][Stage 30:>   (0 + 0) / 8]8]

### Conclusion
In this lab, you explored the use of Apache Spark in smart building monitoring, particularly focusing on HVAC (heating, ventilation, and air conditioning) systems. You now understand the Spark's distributed architecture. You also understand how to simulate real-time sensor data for temperature and humidity, execute SQL queries to identify critical environmental conditions, and output aggregated results for immediate insights.


## Author(s)

Lakshmi Holla

## Other Contributors
Malika Singla
